# RetFound Test

In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning import Trainer
from neural_network.vmamba import VisualMamba

def test_forward():
    batch_size = 2
    img_size = 224
    num_classes = 10
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = VisualMamba(img_size=img_size, num_classes=num_classes, mask_ratio=0.75).to(device)
    x = torch.randn(batch_size, 3, img_size, img_size, device=device)

    logits = model(x)
    print("Logits shape:", logits.shape)
    assert logits.shape == (batch_size, num_classes)

def test_training_step():
    if not torch.cuda.is_available():
        print("⚠️ Skipping training test: Mamba requires CUDA")
        return

    batch_size = 4
    img_size = 224
    num_classes = 10

    x = torch.randn(16, 3, img_size, img_size, device="cuda")
    y = torch.randint(0, num_classes, (16,), device="cuda")
    dataset = TensorDataset(x, y)
    dataloader = DataLoader(dataset, batch_size=batch_size)

    model = VisualMamba(img_size=img_size, num_classes=num_classes, mask_ratio=0.75).cuda()

    trainer = Trainer(max_epochs=1, accelerator="gpu", devices=1, fast_dev_run=True)
    trainer.fit(model, train_dataloaders=dataloader, val_dataloaders=dataloader)

    print("Training loop ran successfully ✅")

if __name__ == "__main__":
    test_forward()
    test_training_step()


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/andre/code/StudentMAE/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
LOCAL_RANK: 

Logits shape: torch.Size([2, 10])



  | Name        | Type       | Params | Mode 
---------------------------------------------------
0 | model       | Sequential | 26.0 K | train
1 | patch_embed | Conv2d     | 147 K  | train
2 | backbone    | Sequential | 6.0 M  | train
3 | norm        | LayerNorm  | 384    | train
4 | head        | Linear     | 1.9 K  | train
---------------------------------------------------
6.2 M     Trainable params
0         Non-trainable params
6.2 M     Total params
24.850    Total estimated model params size (MB)
176       Modules in train mode
0         Modules in eval mode
/home/andre/code/StudentMAE/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/andre/code/StudentMAE/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/d

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.87it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.86it/s]
Training loop ran successfully ✅


# Distillation Test

In [ ]:
# main.py

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer

from distillation.dist import EmbeddingDataset, distill_embeddings
from models.retfound import RETFoundClassifier
from models.vmamba import VisualMamba

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

teacher = RETFoundClassifier(
    checkpoint_path="checkpoints/RETFound_cfp_weights.pth"
).to(device)

student = VisualMamba(
    img_size=224,
    patch_size=16,
    in_chans=3,
    embed_dim=192,
    depth=24,
    learning_rate=1e-4
).to(device)

projector = nn.Linear(192, 1024).to(device)

dummy_data = torch.randn(1000, 3, 224, 224)
embed_loader = DataLoader(EmbeddingDataset(dummy_data), batch_size=32, shuffle=True)

# ---------------- phase 1 ----------------
distill_embeddings(
    teacher=teacher,
    student=student,
    projector=projector,
    dataloader=embed_loader,
    device=device,
    epochs=1,
)

torch.save(student.state_dict(), "vmamba_distilled.pth")

# ================================
# phase 2 lightning finetuning
# ================================

# # freeze backbone — only train heads
# for name, p in student.named_parameters():
#     if not name.startswith("heads."):
#         p.requires_grad_(False)

# # your real labeled dataset
# train_loader = DataLoader(real_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(real_val, batch_size=16)

# trainer = Trainer(
#     max_epochs=10,
#     accelerator="gpu" if torch.cuda.is_available() else "cpu",
# )

# trainer.fit(student, train_loader, val_loader)

# torch.save(student.state_dict(), "vmamba_heads_trained.pth")

Missing keys after loading checkpoint: ['fc_norm.weight', 'fc_norm.bias', 'head.weight', 'head.bias']
[Distill] epoch 1  loss=0.996899


In [4]:
import torch
import torch.nn as nn
from distillation.dist import distill_embeddings, train_head
from models.retfound import RETFoundClassifier
from models.vmamba import VisualMamba
from dataloader.idrid import IDRiDModule
from torchvision import transforms


# ---------------- setup ----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

teacher = RETFoundClassifier(
    checkpoint_path="checkpoints/RETFound_cfp_weights.pth"
).to(device)

student = VisualMamba(
    img_size=224,
    patch_size=16,
    in_chans=3,
    embed_dim=192,
    depth=24,
    learning_rate=1e-4
).to(device)

projector = nn.Linear(192, 1024).to(device)


# ---------------- data ----------------
tfm = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

dm = IDRiDModule(
    root="./data/aaryapatel98/indian-diabetic-retinopathy-image-dataset/versions/1/B.%20Disease%20Grading/B. Disease Grading",
    transform=tfm,
    batch_size=2
)
dm.setup()
train_loader = dm.train_dataloader()
val_loader   = dm.val_dataloader()


# ---------------- phase 1 ----------------
distill_embeddings(
    teacher=teacher,
    student=student,
    projector=projector,
    dataloader=train_loader,
    device=device,
    epochs=1,
)

# ---------------- phase 2 ----------------
# train_head(
#     student=student,
#     dataloader=train_loader,
#     device=device,
#     epochs=1,
# )


# # save student
# student.eval()
# torch.save(student.state_dict(), "./checkpoints/vmamba_distilled.pth")
# print("saved ./checkpoints/vmamba_distilled.pth")

Missing keys after loading checkpoint: ['fc_norm.weight', 'fc_norm.bias', 'head.weight', 'head.bias']
Found 413 images for split=train
Found 103 images for split=test
[Distill] epoch 1  loss=0.545846
